# 004 - CatBoostRegressor & Log Target

## Objectif
Utiliser **CatBoostRegressor** pour exploiter nativement les variables catégorielles (sans OneHotEncoder qui peut perdre de l'information ou créer trop de dimensions).
Toujours avec :
- Transformation Log1p sur la cible.
- Validation croisée (MAE/MAPE).
- Early Stopping pour éviter l'overfitting.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import os

sns.set_style("whitegrid")
%matplotlib inline

## 1. Chargement et Préparation

In [ ]:
TRAIN_PATH = "../Data/train.csv"
TEST_PATH = "../Data/test.csv"

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

X = train_df.drop(columns=["SalePrice"])
y = train_df["SalePrice"]
X_test = test_df

# Identification des colonnes catégorielles
cat_features = list(X.select_dtypes(include=["object"]).columns)
print(f"Categorical features: {len(cat_features)}")

# Préparation pour CatBoost : Conversion en string et remplissage des NaNs catégoriels
for col in cat_features:
    X[col] = X[col].astype(str).fillna("Missing")
    X_test[col] = X_test[col].astype(str).fillna("Missing")

In [ ]:
# Transformation Log de la cible
y_log = np.log1p(y)

## 2. Tuning Léger (Manuel avec CV)

Nous allons tester quelques configurations aléatoires pour trouver de bons hyperparamètres, tout en utilisant l'early stopping sur chaque fold.

In [ ]:
import random

def train_evaluate_cv(params, X, y_log, cat_features):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores = []
    mape_scores = []
    
    # Note: On passe les DataFrame directement à CatBoost
    
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train_log, y_val_log = y_log.iloc[train_index], y_log.iloc[val_index]
        
        train_pool = Pool(X_train, y_train_log, cat_features=cat_features)
        val_pool = Pool(X_val, y_val_log, cat_features=cat_features)
        
        model = CatBoostRegressor(**params, silent=True, allow_writing_files=False)
        model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=200, verbose=False)
        
        # Prédiction (log scale) -> Inverse (real scale)
        preds_log = model.predict(X_val)
        preds = np.expm1(preds_log)
        y_val_true = np.expm1(y_val_log)
        
        mae_scores.append(mean_absolute_error(y_val_true, preds))
        mape_scores.append(mean_absolute_percentage_error(y_val_true, preds))
        
    return np.mean(mae_scores), np.mean(mape_scores)

# Grille de recherche
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.03, 0.05],
    'l2_leaf_reg': [1, 3, 5, 7],
    'random_strength': [1, 2],
    'iterations': [2000] # Suffisant avec early stopping
}

best_mae = float('inf')
best_params = {}

print("Début du tuning (10 essais)...")
random.seed(42)

for i in range(10):
    # Sampling aléatoire
    params = {
        'loss_function': 'MAE',
        'random_seed': 42,
        'iterations': 2000,
        'depth': random.choice(param_grid['depth']),
        'learning_rate': random.choice(param_grid['learning_rate']),
        'l2_leaf_reg': random.choice(param_grid['l2_leaf_reg']),
        'random_strength': random.choice(param_grid['random_strength']),
    }
    
    mae, mape = train_evaluate_cv(params, X, y_log, cat_features)
    print(f"Iter {i+1}: MAE={mae:.2f}, MAPE={mape*100:.2f}%, Params={params}")
    
    if mae < best_mae:
        best_mae = mae
        best_params = params
        
print(f"\nBest MAE: {best_mae:.2f}")
print(f"Best Params: {best_params}")

## 3. Entraînement Final et Soumission

In [ ]:
# On entraîne sur TOUT le train set
# Note : sans eval_set, pas d'early stopping classique. 
# On utilise les paramètres optimaux et on laisse tourner (ou on fixe iterations a une moyenne observée).
full_pool = Pool(X, y_log, cat_features=cat_features)
test_pool = Pool(X_test, cat_features=cat_features)

final_model = CatBoostRegressor(**best_params, silent=True, allow_writing_files=False)
final_model.fit(full_pool)

preds_log = final_model.predict(test_pool)
preds = np.expm1(preds_log)

submission = pd.DataFrame({
    "Id": X_test["Id"],
    "SalePrice": preds
})

submission.head()